In [0]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
#!pip install librosa
import librosa
s, sr=librosa.load('/content/drive/My Drive/data/train_clean_male.wav', sr=None)
S=librosa.stft(s, n_fft=1024, hop_length=512)
sn, sr=librosa.load('/content/drive/My Drive/data/train_dirty_male.wav', sr=None)
X=librosa.stft(sn, n_fft=1024, hop_length=512)

In [0]:
#Now that the network has been trained, let us load the test data

t, sr=librosa.load('/content/drive/My Drive/data/test_x_01.wav', sr=None)
X_test1=librosa.stft(t, n_fft=1024, hop_length=512)

t2, sr=librosa.load('/content/drive/My Drive/data/test_x_02.wav', sr=None)
X_test2=librosa.stft(t2, n_fft=1024, hop_length=512)

In [0]:
mag_X = np.abs(X)
mag_S = np.abs(S)
mag_X_test1 = np.abs(X_test1)
mag_X_test2 = np.abs(X_test2)

trans_mag_X = np.transpose(mag_X)
trans_mag_S = np.transpose(mag_S)
trans_mag_X_test1 = np.transpose(mag_X_test1)
trans_mag_X_test2 = np.transpose(mag_X_test2)

x = tf.placeholder('float',[None,513])
y = tf.placeholder('float',[None,513])

In [0]:
#Question 1

input_layer = tf.reshape(x, shape=[-1, 513, 1])

conv1d_layer1 = tf.layers.conv1d(inputs = input_layer, filters = 16, kernel_size = 16, padding = "SAME", activation = tf.nn.relu)
maxpool1d_layer1 = tf.layers.max_pooling1d(inputs = conv1d_layer1, pool_size = 2, strides = 2)

conv1d_layer2 = tf.layers.conv1d(inputs = maxpool1d_layer1, filters = 32, kernel_size = 8, padding="SAME", activation = tf.nn.relu)
maxpool1d_layer2 = tf.layers.max_pooling1d(inputs = conv1d_layer2, pool_size = 2, strides = 2)

conv_flatten = tf.layers.flatten(maxpool1d_layer2)
  
output = tf.layers.dense(inputs=conv_flatten, units=513, activation=tf.nn.relu)

Instructions for updating:
Use `tf.keras.layers.Conv1D` instead.
Instructions for updating:
Please use `layer.__call__` method instead.
Instructions for updating:
Use keras.layers.MaxPooling1D instead.
Instructions for updating:
Use keras.layers.flatten instead.
Instructions for updating:
Use keras.layers.Dense instead.


In [0]:
def train_neural_network(x):
  
  global final_layer_test1
  global final_layer_test2
  global final_layer_train
  
  prediction = output
  cost = tf.reduce_mean(tf.square(prediction-y))
  optimizer = tf.train.AdamOptimizer(learning_rate=0.001).minimize(cost)
  hm_epochs = 100
  
  with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    for epoch in range(hm_epochs):
      
      epoch_loss = 0
      
      for _ in range(int(trans_mag_X.shape[0]/100)):
        start = _*100
        end = (_*100) + (100-1)
        epoch_x = trans_mag_X[start:end,:]
        epoch_y = trans_mag_S[start:end,:]
        _, c = sess.run([optimizer,cost],feed_dict = {x: epoch_x, y: epoch_y})
        
        epoch_loss += c

      print('Epoch', epoch+1, 'completed out of', hm_epochs, 'loss:', epoch_loss)
    
    final_layer_test1 = sess.run(prediction,feed_dict = {x:trans_mag_X_test1})
    final_layer_test2 = sess.run(prediction,feed_dict = {x:trans_mag_X_test2})
    final_layer_train = sess.run(prediction,feed_dict = {x:trans_mag_X})
      
train_neural_network(x)

Epoch 1 completed out of 100 loss: 1.696570586413145
Epoch 2 completed out of 100 loss: 1.218347080051899
Epoch 3 completed out of 100 loss: 0.7938978178426623
Epoch 4 completed out of 100 loss: 0.5679759415797889
Epoch 5 completed out of 100 loss: 0.45821112347766757
Epoch 6 completed out of 100 loss: 0.3924139169976115
Epoch 7 completed out of 100 loss: 0.3500605756416917
Epoch 8 completed out of 100 loss: 0.32819020142778754
Epoch 9 completed out of 100 loss: 0.2965543402824551
Epoch 10 completed out of 100 loss: 0.27049294323660433
Epoch 11 completed out of 100 loss: 0.25296554900705814
Epoch 12 completed out of 100 loss: 0.22713872767053545
Epoch 13 completed out of 100 loss: 0.21671845763921738
Epoch 14 completed out of 100 loss: 0.21117949346080422
Epoch 15 completed out of 100 loss: 0.2010033444967121
Epoch 16 completed out of 100 loss: 0.20338364993222058
Epoch 17 completed out of 100 loss: 0.19186382414773107
Epoch 18 completed out of 100 loss: 0.18095195316709578
Epoch 19 co

In [0]:
S_test1 = X_test1/mag_X_test1
S_test1 = np.multiply(S_test1,np.transpose(final_layer_test1))

S_test2 = X_test2/mag_X_test2
S_test2 = np.multiply(S_test2,np.transpose(final_layer_test2))

S_train = X/mag_X
S_train = np.multiply(S_train,np.transpose(final_layer_train))

In [0]:
s_test1 = librosa.istft(S_test1, hop_length=512)
librosa.output.write_wav('1d_test_s_01_reconstruct.wav', s_test1, sr)

s_test2 = librosa.istft(S_test2, hop_length=512)
librosa.output.write_wav('1d_test_s_02_reconstruct.wav', s_test2, sr)

s_train = librosa.istft(S_train, hop_length=512)
librosa.output.write_wav('1d_train_reconstruct.wav', s_train, sr)

In [0]:
size = np.shape(s_train)[0]
s = s[: size]
num = np.dot(np.transpose(s),s)
den = np.dot(np.transpose(s - s_train),(s - s_train))
SNR = 10 * np.log10(num/den)
print('Value of SNR : ' + str(SNR))

Value of SNR : 14.344884157180786


In [0]:
#Question 2

x = tf.placeholder('float',[None,20,513])
y = tf.placeholder('float',[None,513])

In [0]:
def scan(z):
  new_list = []
  for i in range(0,len(z)-19):
    new_list.append(z[i:i+20,:])
  return new_list

In [0]:
input_layer = tf.reshape(x, shape=[-1, 20, 513, 1])

conv2d_layer1 = tf.layers.conv2d(inputs = input_layer, filters = 16, kernel_size = [4,4], strides = [1,1], padding = "SAME", activation = tf.nn.relu)
maxpool2d_layer1 = tf.layers.max_pooling2d(inputs = conv2d_layer1, pool_size = [2,2], strides = [2,2])

conv2d_layer2 = tf.layers.conv2d(inputs = maxpool2d_layer1, filters = 32, kernel_size = [2,2], strides = [1,1], padding="SAME", activation = tf.nn.relu)
maxpool2d_layer2 = tf.layers.max_pooling2d(inputs = conv2d_layer2, pool_size = [2,2], strides = [2,2])

conv_flatten = tf.layers.flatten(maxpool2d_layer2)
  
output1 = tf.layers.dense(inputs=conv_flatten, units=1024, activation=tf.nn.relu)

drop_out = tf.nn.dropout(output1,rate=0.2)

output = tf.layers.dense(inputs=drop_out, units=513, activation=tf.nn.relu)

Instructions for updating:
Use `tf.keras.layers.Conv2D` instead.
Instructions for updating:
Use keras.layers.MaxPooling2D instead.


In [0]:
new_trans_mag_X = scan(trans_mag_X)
new_trans_mag_X_test1 = scan(trans_mag_X_test1)
new_trans_mag_X_test2 = scan(trans_mag_X_test2)

new_trans_mag_S = trans_mag_S[19:,:]
new_trans_mag_X = np.reshape(new_trans_mag_X , (len(new_trans_mag_X) , 20 , 513))
new_trans_mag_X_test1 = np.reshape(new_trans_mag_X_test1 , (len(new_trans_mag_X_test1) , 20 , 513))
new_trans_mag_X_test2 = np.reshape(new_trans_mag_X_test2 , (len(new_trans_mag_X_test2) , 20 , 513))


In [0]:
cost = tf.reduce_mean(tf.square(output-y))
optimizer = tf.train.AdamOptimizer(learning_rate=0.001).minimize(cost)
hm_epochs = 100

with tf.Session() as sess:
  sess.run(tf.global_variables_initializer())

  for epoch in range(hm_epochs):

    epoch_loss = 0

    for _ in range(int(len(new_trans_mag_X)/128)):
      start = _*128
      end = (_*128) + (128-1)
      epoch_x = new_trans_mag_X[start:end]
      epoch_y = new_trans_mag_S[start:end]
      
      epoch_x = epoch_x.reshape(np.shape(epoch_x)[0] , np.shape(epoch_x)[1] , np.shape(epoch_x)[2])
      
      _, c = sess.run([optimizer,cost],feed_dict = {x: epoch_x, y: epoch_y})

      epoch_loss += c

    print('Epoch', epoch+1, 'completed out of', hm_epochs, 'loss:', epoch_loss)
  
  final_layer_test1 = sess.run(output,feed_dict = {x:new_trans_mag_X_test1})
  final_layer_test2 = sess.run(output,feed_dict = {x:new_trans_mag_X_test2})
  final_layer_train = sess.run(output,feed_dict = {x:new_trans_mag_X})
      


Epoch 1 completed out of 100 loss: 1.922581534832716
Epoch 2 completed out of 100 loss: 1.437642678618431
Epoch 3 completed out of 100 loss: 1.2326178010553122
Epoch 4 completed out of 100 loss: 1.060838183388114
Epoch 5 completed out of 100 loss: 0.9023817908018827
Epoch 6 completed out of 100 loss: 0.7676181001588702
Epoch 7 completed out of 100 loss: 0.6792701613157988
Epoch 8 completed out of 100 loss: 0.6177981644868851
Epoch 9 completed out of 100 loss: 0.5452281879261136
Epoch 10 completed out of 100 loss: 0.4892150992527604
Epoch 11 completed out of 100 loss: 0.45857666805386543
Epoch 12 completed out of 100 loss: 0.42620926443487406
Epoch 13 completed out of 100 loss: 0.40404571406543255
Epoch 14 completed out of 100 loss: 0.40127144381403923
Epoch 15 completed out of 100 loss: 0.3851292347535491
Epoch 16 completed out of 100 loss: 0.3848615000024438
Epoch 17 completed out of 100 loss: 0.35299691604450345
Epoch 18 completed out of 100 loss: 0.35740442015230656
Epoch 19 complet

In [0]:
silent_frames = np.empty([19,513])
silent_frames.fill(1e-10)

In [0]:
S_test1 = X_test1/mag_X_test1
S_test1 = np.multiply(S_test1,np.vstack((silent_frames,final_layer_test1)).T)

S_test2 = X_test2/mag_X_test2
S_test2 = np.multiply(S_test2,np.vstack((silent_frames,final_layer_test2)).T)

S_train = X/mag_X
S_train = np.multiply(S_train,np.vstack((silent_frames,final_layer_train)).T)


In [0]:
s_test1 = librosa.istft(S_test1, hop_length=512)
librosa.output.write_wav('2d_test_s_01_reconstruct.wav', s_test1, sr)

s_test2 = librosa.istft(S_test2, hop_length=512)
librosa.output.write_wav('2d_test_s_02_reconstruct.wav', s_test2, sr)

s_train = librosa.istft(S_train, hop_length=512)
librosa.output.write_wav('2d_train_reconstruct.wav', s_train, sr)

In [0]:
size = np.shape(s_train)[0]
s_train = s_train[: s.shape[0]]
num = np.dot(np.transpose(s),s)
den = np.dot(np.transpose(s - s_train),(s - s_train))
SNR = 10 * np.log10(num/den)
print('Value of SNR : ' + str(SNR))

Value of SNR : 11.303973197937012
